# Pinecone - Creando un asistente conversacional

### Arquitectura RAG

## Introducción <a name="intro"></a>

El propósito general de este notebook es generar un asistente conversacional basado en la arquitectura RAG (_Retrieval Augmented Generation_).

1. Dispondremos de unos documentos PDFs que serán nuestra base de conocimiento, los cuáles vectorizaremos y almacenaremos como Embeddings en un índice de Pinecone
2. Posteriormente, a través de LangChain podremos lanzar queries y que automáticamente se pasen por el modelo de embedding y se conecten al índice de Pinecone para hacer una búsqueda por similitud, para posteriormente pasarle los trozos más relevantes al LLM para que nos devuelva una respuesta.

## LangChain 🦜 <a name="langchain"></a>

__LangChain__ es un marco para desarrollar aplicaciones basadas en modelos del lenguaje (únicamente se especializa en NLP)

Para instalar LangChain en Python haremos:

```python
!pip install langchain
```

Además de permitirnos encadenar conversaciones, LangChain tiene distintas funciones para leer archivos y hacer las divisiones de los textos en chunks

El módulo `document_loaders` https://python.langchain.com/docs/modules/data_connection/document_loaders/ tiene la capacidad de cargar los siguientes tipos de archivos:
+ CSV
+ Directorios
+ PDF
+ Markdown y texto
+ HTML
+ JSON

Importamos un archivo PDF, previamente necesitamos instalar una dependencia
```python
!pip install pypdf
```

NOTA: Con la función `PyPDFDirectoryLoader` pueden cargarse todos los PDFs almancenados en un directorio, si se quiere cargar un único documento puede emplearse la función `PyPDFLoader`


In [ ]:
# Instalar en el entorno
#%pip install numpy
#%pip install langchain
#%pip install langchain-community
#%pip install pypdf
#%pip install sentence_transformers
#%pip install langchain_pinecone
#%pip install langchain_core
#%pip install langchain_huggingface
#%pip install huggingface-hub
#%pip install langchain_openai

In [2]:
import numpy as np
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
from dotenv import load_dotenv, find_dotenv
from langchain_huggingface import HuggingFaceEndpoint
from langchain.chains import RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

/Applications/anaconda3/envs/chatbot/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
ruta_docs = "docs_ejemplo"

In [4]:
# Abrimos la ruta en la que se encuentran los PDF
loader = PyPDFDirectoryLoader(ruta_docs)

# Cargamos el PDF 
raw_pdfs = loader.load()

# Vemos que contiene nuestro archivo
raw_pdfs[: 10]

[Document(metadata={'source': 'docs_ejemplo/MASTER_INDEX.pdf', 'page': 0}, page_content='Contenido \nMáster en Big Data ................................ ................................ ...........................  3 \nMÓDULO 1 - Fundamentos de tratamiento de datos para Data Science ............ 3 \nMÓDULO 2 - Business intelligence ................................ ................................ . 5 \nMÓDULO 3 - Aprendizaje Automático Aplicado (Machine Learning) .................. 8 \nMÓDULO 4 - Minería de Texto y Procesamiento del Lenguaje Natural (PLN) ..... 10 \nMÓDULO 5 - Inteligencia de Negocio y Visualización ................................ ..... 12 \nMÓDULO 6 - Infraestructura Big Data ................................ ...........................  15 \nMÓDULO 7 - Almacenamiento e Integración de Datos ................................ ... 18 \nMÓDULO 8 - Valor y Contexto de la Analítica Big Data ................................ ... 20 \nMÓDULO 9 - Aplicaciones Analíticas. Casos p

In [5]:
print("Total de elementos cargados --> ", len(raw_pdfs))

Total de elementos cargados -->  58


In [6]:
print(raw_pdfs[20].page_content)


El contexto económico actual destaca la importancia del Big Data para el 
crecimiento empresarial. El valor del dato radica en su capacidad de generar 
información relevante y estratégica. Existen distintos tipos de datos, desde los 
estructurados hasta los no estructurados, que cada empresa puede utilizar desde 
una perspectiva de negocio. Los tipos de analítica se dividen en descriptiva, 
predictiva y prescriptiva, cada una aportando distintos niveles de conocimiento. La 
transformación hacia una organización a nalítica requiere puntos clave como la 
cultura de datos y una infraestructura adecuada. El Big Data ayuda a las empresas 
a ser más competitivas y a responder mejor a las demandas del mercado. 
2. Proyectos de big data 
o Factores clave 
o Perfiles 
o Áreas de aplicación 
o Fases de un proyecto Big Data 
o Entornos en un proyecto Big Data 
o Proyectos de datos 
Un proyecto de Big Data requiere considerar varios factores clave, como la 
infraestructura tecnológica y el talento

Vemos que se ha cargado un documento con todas las páginas. No obstante, no podemos pasarle toda la información de golpe al LLM (por el límite de tokens), sino solamente el trozo más similar. El primer paso es dividir el texto en trozos mediante técnicas de _chunking_.

Para ir dividiendo la información del archivo PDF en pequeños fragmentos volvemos a emplear funciones de LangChain: `RecursiveCharacterTextSplitter()`: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/


In [7]:
# Divide en trozos que no superen el maximo de chunk_size, intentando mantener un sentido
# semantico mediante el parametro separators (no cortar en medio de una frase)
# Despues, añade contexto de chunks anteriores mediante el parametro overlap

text_splitter = RecursiveCharacterTextSplitter(

    separators = ["\n\n","\n",".",",", ";", " "], # 
    
    # Esto asegura que los fragmentos no excedan el límite de 1000 caracteres
    # el tamaño de los chunks puede ser inferior si el corte debe ocurrir en un separador
    chunk_size = 1000,
    
    # El solapamiento es crucial para mantener el contexto entre fragmentos: evita que se pierda información relevante
    chunk_overlap = 300,
    
    # Función de longitud que se utiliza para calcular el tamaño del texto.
    length_function = len,
    
    # `add_start_index = True` permite saber desde qué punto del texto original se originó cada fragmento,
    # lo cual es útil si luego necesitas mapear los resultados generados a la fuente original del documento.
    add_start_index = True,
)

# Dividimos el archivo en fragmentos (chunks)
docs_split = text_splitter.split_documents(raw_pdfs)

In [8]:
print("Número de Chunks producidos desde nuestro PDF --> ", len(docs_split))

Número de Chunks producidos desde nuestro PDF -->  154


In [9]:
docs_split[20]

Document(metadata={'source': 'docs_ejemplo/MASTER_INDEX.pdf', 'page': 7, 'start_index': 0}, page_content='o Inteligencia de cliente \no Ingesta de datos CRM \nLos sistemas CRM (Customer Relationship Management) son esenciales para \ngestionar las relaciones con los clientes y analizar sus comportamientos y \npreferencias. La inteligencia de cliente se refiere al análisis profundo de estos datos \npara entender mejor a  los clientes y ofrecerles experiencias personalizadas. La \ningesta de datos CRM implica la recopilación de información de múltiples fuentes \npara alimentar el sistema y generar estrategias de marketing más efectivas y \nfocalizadas. \nMÓDULO 3 - Aprendizaje Automático Aplicado (Machine Learning) \n1. Introducción al aprendizaje automático \no El proceso de la minería de datos \no Tipos de aprendizaje automático \no Introducción a SCIKIT-LEARN y THEANO \no Uso básico de un modelo \nEl aprendizaje automático es una rama de la inteligencia artificial que se centra en \nel

In [10]:
print(docs_split[20].page_content)

o Inteligencia de cliente 
o Ingesta de datos CRM 
Los sistemas CRM (Customer Relationship Management) son esenciales para 
gestionar las relaciones con los clientes y analizar sus comportamientos y 
preferencias. La inteligencia de cliente se refiere al análisis profundo de estos datos 
para entender mejor a  los clientes y ofrecerles experiencias personalizadas. La 
ingesta de datos CRM implica la recopilación de información de múltiples fuentes 
para alimentar el sistema y generar estrategias de marketing más efectivas y 
focalizadas. 
MÓDULO 3 - Aprendizaje Automático Aplicado (Machine Learning) 
1. Introducción al aprendizaje automático 
o El proceso de la minería de datos 
o Tipos de aprendizaje automático 
o Introducción a SCIKIT-LEARN y THEANO 
o Uso básico de un modelo 
El aprendizaje automático es una rama de la inteligencia artificial que se centra en 
el desarrollo de algoritmos que permiten a las computadoras aprender de los datos


In [11]:
len(docs_split[20].page_content)

957

## Embeddings <a name="embeddings"></a>

Una vez que tenemos separado en pequeños fragmentos nuestro documento PDF, ya podemos obtener los vectores de Word Embeddings sobre el mismo, para posteriormente, poder almacenarlos en Pinecone. 

Para buscar el modelo adecuado podemos investigar dentro de HuggingFace modelos que soporten creación de Embeddings con sentence similarity: https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending

En nuestro caso, vamos a cargar uno de los más utilizados `sentence-transformers/all-MiniLM-L6-v2` https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Este modelo tiene una dimensionalidad de 384.

Para poder trabajar con los modelos de sentence-transformers debemos instalar previamente el paquete:
```python
!pip install sentence_transformers
```

In [12]:
# Cargamos el modelo de embeddings

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}
)

In [13]:
# Comprobamos cómo obtiene Embeddings automáticamente
#  desde una página cualquiera de nuestro documento.
print(docs_split[100].page_content)

con las operaciones de TI para facilitar la implementación, monitoreo y 
mantenimiento de modelos en producción. Los requerimientos de MLOps incluyen 
la automatización del proceso de d esarrollo y la integración continua. MLOps en 
Azure se implementa a través de herramientas como Azure Machine Learning, que 
facilitan la colaboración y el monitoreo de modelos. AWS ofrece SageMaker, una 
solución integral para gestionar el ciclo de vida de los modelos de machine learning. 
En Google, MLOps se gestiona mediante Vertex AI, que permite a los equipos de 
datos crear y desplegar modelos de forma eficiente y escalable. 
MÓDULO 8 - Series temporales y modelos prescriptivos. 
Optimización. Modelos de grafos 
1. Optimización 
o Descripción de la optimización matemática 
o Programación lineal 
o Programación entera 
o Programación no lineal 
o Heurísticas y metaheurísticas 
o Optimización bajo incertidumbre 
o Optimización y machine learning


In [14]:
huggingface_embeddings.embed_query(docs_split[100].page_content) 

[-0.08006897568702698,
 -0.028031377121806145,
 -0.06092718988656998,
 -0.028301836922764778,
 0.002666361164301634,
 -0.025970494374632835,
 -0.017113879323005676,
 0.024322504177689552,
 -0.0425584614276886,
 0.06241614371538162,
 -0.0049565932713449,
 0.02623717114329338,
 0.02730262652039528,
 -0.012406567111611366,
 -0.008839382790029049,
 0.0734325647354126,
 0.00457391794770956,
 -0.035690370947122574,
 -0.0934627503156662,
 -0.010611403733491898,
 0.11288634687662125,
 -0.11038278788328171,
 -0.10694729536771774,
 0.035126861184835434,
 -0.015215888619422913,
 -0.0487988106906414,
 -0.0037630328442901373,
 0.03400614112615585,
 -0.014293663203716278,
 -0.04002578556537628,
 -0.02971450611948967,
 -0.02808339335024357,
 0.1056986004114151,
 0.05695825815200806,
 0.009663194417953491,
 0.04797649011015892,
 0.04284169524908066,
 -0.06265434622764587,
 -0.06185232475399971,
 0.006601959001272917,
 -0.02214927226305008,
 -0.09500030428171158,
 -0.046009842306375504,
 -0.10021927207

## Carga de los Embeddings en Pinecone <a name="persist"></a> 

Para insertar documentos en Pinecone vuelve a ayudarnos LangChain, ya que tiene integraciones directas con múltiples bases de datos vectoriales https://python.langchain.com/docs/modules/data_connection/vectorstores/

```python
pip install langchain-core 
pip install langchain-pinecone
```

La función que vamos a utilizar nos permitirá, de una sola vez (1) convertir los textos a embedding (con el modelo cargado antes) y (2) subir los embeddings al índice de Pinecone

**NOTA**: En este punto, debo tener creado un índice en Pinecone con las dimensiones requeridas para mi modelo de embedding (en este caso, 384) + haber guardado el nombre del nuevo índice en el fichero .env

In [15]:
load_dotenv(find_dotenv())

True

In [16]:
# Esta funcion permitira pasar cada documento por el embedding y cargarlo en el indice

docs = PineconeVectorStore.from_documents(
    documents  = docs_split, 
    embedding  = huggingface_embeddings, 
    index_name = os.environ["INDEX_CHATBOT"]
    )

## LLM y generación de prompts <a name="hf"></a> 

Lo primero que deberemos conseguir será la API TOKEN de Huggingface. Posteriormente, empleamos la función `HuggingFaceHub` a la cuál le pasaremos principalmente como parámetro (en `repo_id`) el nombre del modelo. Para poder obtener toda la lista de modelos públicos de Huggingface, demos seleccionar en Models -> Natural Language Processing - Text Generation.

En nuestro caso, vamos a usar una de las mejores propuestas en cuanto a modelos públicos __Mistral__ https://huggingface.co/mistralai con el modelo `Mistral-7B-Instruct-v0.2`. https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

Importante: Debemos aceptar los términos de licencia para poder utilizar el modelo. 

Guardamos la API KEY de hugging face en nuestro fichero .env

In [17]:
load_dotenv(find_dotenv())

True

In [18]:
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm = HuggingFaceEndpoint(    
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    repo_id="mistralai/Mistral-7B-Instruct-v0.2", #"meta-llama/Llama-3.2-1B", #"meta-llama/Llama-3.2-3B-Instruct"
    temperature = 0.5, # a mayor temperatura, mayor creatividad y menos conservador
    model_kwargs={"max_length":2048})
                  #"max_new_tokens": 500})

Una vez definido nuestro LLM, podemos hacer una pequeña prueba, para pasarle una consulta (prompt) al llm utilizaremos funciones como `run` o `invoke`

Antes de usar nuestra BBDD vectorial como contexto, hacemos una prueba solo con el LLM (responderá en base a la información con la que fue entrenado)

In [19]:
query = """
        Hola, dame las características de Python 
        """ 
llm.client.api_url = "mistralai/Mistral-7B-Instruct-v0.2" # hay veces que pierde donde está el repo (no siempre es necesario pero se hace por si acaso)

In [20]:
print(llm.invoke(query)) # despues regularemos el tamaño del mensaje de salida

1. Python es una lenguaje de programación de alto nivel, interpretado y de sintaxis limpia.
        2. Es ampliamente utilizado en la web, ciencia de datos, automatización y muchas otras áreas.
        3. Es altamente portátil y puede ser ejecutado en muchas plataformas diferentes, como Windows, MacOS, Linux, etc.
        4. Python es conocido por su sintaxis simple y clara, lo que lo hace ideal para principiantes y expertos.
        5. Python tiene una amplia biblioteca estándar y una comunidad activa de desarrolladores que crean paquetes adicionales.
        6. Es altamente escalable y se puede usar para desarrollar aplicaciones pequeñas y grandes.
        7. Python es dinámico, lo que significa que puede modificar el programa en ejecución.
        8. Python tiene un buen soporte para multihilo y multiprocesamiento, lo que lo hace ideal para aplicaciones que requieren mucha computación.
        9. Python tiene un buen soporte para integración con otros lenguajes y herramientas, lo qu

Como vemos, la respuesta es genérica ya que todavía no estamos integrando nuestra base de conocimiento.

Para interactuar con nuestra base de conocimento debemos elaborar una función _retriever_, que será capaz de buscar por similaridad documentos (en nuestro caso, a través de los documentos ya cargados en Pinecone)

In [21]:
# Cargamos el vector store
vectorstore = PineconeVectorStore(
    index_name=os.environ["INDEX_CHATBOT"],
    pinecone_api_key=os.environ["PINECONE_API_KEY"],
    embedding=huggingface_embeddings,
)


# Crear el retriever a partir del VectorStore
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 3} # Consultas basadas en los 3 mejores resultados
    )

Creamos el Prompt, donde no solo hay que indicarle la pregunta sino todas las instrucciones que debe tener en cuenta (contexto a utilizar, memoria, si quieres que no invente información...)

In [22]:
prompt_template = """Eres un comercial especializado de una escuela de negocios que asesora a futuros alumnos sobre másters.
Contesta la pregunta basandote en el contexto (delimitado por <ctx> </ctx>) y en el histórico del chat (delimitado por <hs></hs>) de abajo.
1. Da una respuesta lo más concisa posible.
2. Si la respuesta no aparece en el contexto proporcionado di que no tienes la información.
3. Limítate a responder a la pregunta y proporciona solo la respuesta útil

Información proporcionada
-------
<ctx>
Contexto: {context}
</ctx>
-------
<hs>
Histórico: {chat_history}
</hs>
-------
Pregunta: {question}
Respuesta útil:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context",  "chat_history", "question"]
)

Ahora, ya solo nos queda definir nuestro chatbot a través de la función que debe recibir el prompt, el llm y el objeto retriever `RetrievalQA.from_chain_type()`. QA es para preguntas y respuestas, no obstante, hay otro tipo de cadenas para por ejemplo devolver código.
Al asistente, habrá que pasarle tanto la pregunta como el parámetro opcional `memory`, que definiremos previamente con la función `ConversationBufferMemory`


In [23]:
memory = ConversationBufferMemory(
        memory_key="chat_history",
       input_key="question"
)

retrievalQA = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": PROMPT,
               "memory": memory})

/var/folders/ms/b0wp19xj6t95pygkb1yqjh2c0000gn/T/ipykernel_3822/1321285784.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [32]:
query = "en qué módulo puedo aprender business intelligence?"

respuesta = retrievalQA.invoke({"query": query})
print(respuesta['result'])

El Business Intelligence (BI) se aborda en el módulo 2 de la estructura de estudios proporcionada. En este módulo, se introduce la concepto de BI, su importancia, los componentes de los sistemas de BI, y los tipos de análisis que se pueden realizar. Además, se menciona que Pandas es una herramienta importante para la manipulación y análisis de datos estructurados en el contexto de BI. Por lo tanto, el módulo 2 es el lugar adecuado para aprender sobre Business Intelligence.


In [33]:
respuesta # tambien devuelve los trozos de documento mas similares

{'query': 'en qué módulo puedo aprender business intelligence?',
 'result': 'El Business Intelligence (BI) se aborda en el módulo 2 de la estructura de estudios proporcionada. En este módulo, se introduce la concepto de BI, su importancia, los componentes de los sistemas de BI, y los tipos de análisis que se pueden realizar. Además, se menciona que Pandas es una herramienta importante para la manipulación y análisis de datos estructurados en el contexto de BI. Por lo tanto, el módulo 2 es el lugar adecuado para aprender sobre Business Intelligence.',
 'source_documents': [Document(id='751b498a-dbc2-41f9-9dce-5553c954ed39', metadata={'page': 12.0, 'source': 'docs_ejemplo/MASTER_INDEX.pdf', 'start_index': 0.0}, page_content='o Estructura de un sistema de business intelligence \no Datos, información y conocimiento \no Alfabetización de datos \nEl business intelligence (BI) se refiere al conjunto de procesos, tecnologías y \nherramientas que convierten los datos en información útil para ap

In [34]:
query = "en ese modulo se estudian bases de datos?"

respuesta = retrievalQA.invoke({"query": query})

print(respuesta['result'])

Sí, el módulo 2 de la estructura de estudios incluye el tema de la ciencia de datos, donde se abordan técnicas de análisis, minería y visualización de datos. Dentro de este tema, se estudian diferentes tipos de bases de datos, incluyendo bases de datos relacionales y NoSQL, así como sus respectivas características y aplicaciones.


Para obtener los documentos utilizados para elaborar la respuesta, podemos crear una función que nos extraiga de forma estructurada la página y el texto obtenido

In [35]:
def obtener_doc_sources(respuesta):

    lista_docs = respuesta['source_documents']

    documentos = ["Página: " + str(elemento.metadata['page']) + " - Contenido: " + elemento.page_content for elemento in lista_docs]

    return documentos

In [36]:
obtener_doc_sources(respuesta)

['Página: 44.0 - Contenido: presentaciones y análisis exploratorios en los que se busca una mayor interacción \ncon los datos. \nMÓDULO 2 - La ciencia de datos. Técnicas de análisis, minería y \nvisualización \n1. El ciclo de vida del dato \no Definición de ciencia de datos \no El ciclo de vida de los datos \no Definición de objetivos en un proyecto de datos \no Identificación de los datos necesarios \no Preparación y preproceso \no Análisis y modelado \no Validación y prueba \nLa ciencia de datos se refiere al uso de técnicas y herramientas para extraer \ninformación y conocimiento de los datos. El ciclo de vida del dato comienza con la \ndefinición de los objetivos de un proyecto de datos, lo que implica entender el \nproblema a reso lver y los resultados esperados. Luego, se identifican los datos \nnecesarios para cumplir esos objetivos. La preparación y preproceso de datos \nincluyen la limpieza, transformación y estructuración de los datos para su análisis.',
 'Página: 17.0 - Cont

In [37]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='en qué módulo puedo aprender business intelligence?', additional_kwargs={}, response_metadata={}), AIMessage(content='El Business Intelligence (BI) se aborda en el módulo 2 de la estructura de estudios proporcionada. En este módulo, se introduce la concepto de BI, su importancia, los componentes de los sistemas de BI, y los tipos de análisis que se pueden realizar. Además, se menciona que Pandas es una herramienta importante para la manipulación y análisis de datos estructurados en el contexto de BI. Por lo tanto, el módulo 2 es el lugar adecuado para aprender sobre Business Intelligence.', additional_kwargs={}, response_metadata={}), HumanMessage(content='en ese modulo se estudian bases de datos?', additional_kwargs={}, response_metadata={}), AIMessage(content='Sí, el módulo 2 de la estructura de estudios incluye el tema de la ciencia de datos, donde se abordan técnicas de análisis, minería y visualización de datos. Dentro de 

In [38]:
memory.clear() # para resetear memoria (si no se hace, se puede superar el limite de tokens y da error)

In [39]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[])